In [1]:
import pandas as pd
import sqlite3
from sys import path
path.append(r"\\192.168.40.46\0. Sharing Folder\my-modules")
import Function_data_analyst as mf
import cx_Oracle

In [2]:
db_nice =cx_Oracle.connect(user="NICEDB",password="NiceVmg368",dsn="192.168.18.32:1521/NICEDB2")
db_brandnamebhxh = sqlite3.connect("db_brandnamebhxh.db")

In [12]:
standard_cols = ["PHONE", "MABHXH","LASTEST_MONTH"]
def get_pair_brandname(from_db, from_table,to_db, to_table,sql, chunksize):
    df_chunk = mf.read_data_from_db(from_db,from_table, sql_query= sql, chunksize=chunksize)
    stt=0
    for chunk in df_chunk:
        stt+=1
        chunk.columns = standard_cols
        chunk = mf.DFCleaner(chunk , strip_cols = standard_cols)
        chunk.to_sql(to_table,to_db,if_exists="append",index=False)
        print("done to rows :",stt*chunksize,end="\r")

In [4]:
from_table= "BRANDNAME_SI_INDIVIDUAL"
sql = """
SELECT USER_ID, MABHXH, max(MONTH)
FROM {}
GROUP BY USER_ID, MABHXH
""".format(from_table)
get_pair_brandname(from_db=db_nice, from_table=from_table,
                   to_db=db_brandnamebhxh, to_table="BHXH_PHONE_BRANDNAME",
                   sql=sql, chunksize=100000)

Bảng BRANDNAME_SI_INDIVIDUAL offset 0 dòng, all records, chunksize 100000


In [5]:
from_table= "BRANDNAME_SI_LEAVE"
sql = """
SELECT USER_ID, MABHXH, max(MONTH)
FROM {}
GROUP BY USER_ID, MABHXH
""".format(from_table)
get_pair_brandname(from_db=db_nice, from_table=from_table,
                   to_db=db_brandnamebhxh, to_table="BHXH_PHONE_BRANDNAME",
                   sql=sql, chunksize=100000)

Bảng BRANDNAME_SI_LEAVE offset 0 dòng, all records, chunksize 100000


In [6]:
from_table= "BRANDNAME_SI_8079"
sql = """
SELECT USER_ID, MABHXH_FIXED, max(MONTH)
FROM {}
GROUP BY USER_ID, MABHXH_FIXED
""".format(from_table)
get_pair_brandname(from_db=db_nice, from_table=from_table,
                   to_db=db_brandnamebhxh, to_table="BHXH_PHONE_BRANDNAME",
                   sql=sql, chunksize=100000)

Bảng BRANDNAME_SI_8079 offset 0 dòng, all records, chunksize 100000


In [8]:
from_table= "BRANDNAME_HEALTH_INSURANCE_8079"
sql = """
SELECT USER_ID, MABHXH, max(MONTH)
FROM {}
GROUP BY USER_ID, MABHXH
""".format(from_table)
get_pair_brandname(from_db=db_nice, from_table=from_table,
                   to_db=db_brandnamebhxh, to_table="BHXH_PHONE_BRANDNAME",
                   sql=sql, chunksize=100000)


Bảng BRANDNAME_HEALTH_INSURANCE_8079 offset 0 dòng, all records, chunksize 100000


In [13]:
from_table= "BRANDNAME_HEALTH_INSURANCE"
sql = """
SELECT USER_ID, MABHXH, max(MONTH)
FROM (select USER_ID, SUBSTR(MABHYT,-10) as MABHXH, MONTH from {})
GROUP BY USER_ID, MABHXH
""".format(from_table)
       
get_pair_brandname(from_db=db_nice, from_table=from_table,
                   to_db=db_brandnamebhxh, to_table="BHXH_PHONE_BRANDNAME",
                   sql=sql, chunksize=100000)

Bảng BRANDNAME_HEALTH_INSURANCE offset 0 dòng, all records, chunksize 100000


In [15]:
from_table= "TBL_SI_EXTRACT_BRANDNAME"
sql = """
SELECT USER_ID, MABHXH, max(MONTH) FROM (
(select USER_ID, SUBSTR(column_9,-10) as MABHXH, MONTH from {from_table} where table_name = 'tbl_bhxh' and column_9 is not null)
union 
(select USER_ID, column_2 as MABHXH, MONTH from {from_table} where table_name = 'tbl_bhxh' and column_2 is not null)
)
GROUP BY USER_ID, MABHXH
""".format(from_table=from_table)
get_pair_brandname(from_db=db_nice, from_table=from_table,
                   to_db=db_brandnamebhxh, to_table="BHXH_PHONE_BRANDNAME",
                   sql=sql, chunksize=100000)

Bảng TBL_SI_EXTRACT_BRANDNAME offset 0 dòng, all records, chunksize 100000


In [44]:
from_table= "TBL_SI_EXTRACT_BRANDNAME"
sql = """
select * from
(select USER_ID, SUBSTR(column_9,-10) as MABHXH, MONTH from {from_table} where table_name = 'tbl_bhxh' and column_9 is not null)
union 
(select USER_ID, column_2 as MABHXH, MONTH from {from_table} where table_name = 'tbl_bhxh' and column_2 is not null)

""".format(from_table=from_table)
df = mf.read_data_from_db(db_nice,from_table, sql_query= sql,n_records=10)
df

Bảng TBL_SI_EXTRACT_BRANDNAME offset 0 dòng, 10 records


,USER_ID,MABHXH,MONTH
0,325252067,3110071352,202108
1,325252067,3110071352,202109
2,325273560,5620535869,202109
3,325273560,5621390810,202109
4,325280576,3020514606,202107
...,...,...,...
7076826,989972261,7932442389,202107
7076827,989973362,8621587138,202107
7076828,989982647,5620438961,202109
7076829,989982647,5620439663,202108


In [24]:
standard_cols = ["PHONE", "MABHXH","LASTEST_MONTH"]
from_table= "BHXH_PHONE_BRANDNAME"
sql = """
SELECT PHONE, MABHXH, max(LASTEST_MONTH)
FROM {}
GROUP BY PHONE, MABHXH
""".format(from_table)
get_pair_brandname(from_db=db_brandnamebhxh, from_table="BHXH_PHONE_BRANDNAME",
                   to_db=db_brandnamebhxh, to_table="BHXH_PHONE_BRANDNAME_FINAL",
                   sql=sql, chunksize=100000)

Bảng BHXH_PHONE_BRANDNAME offset 0 dòng, all records, chunksize 100000


In [25]:
df_chunk=pd.read_sql_query("select distinct MABHXH from  BHXH_PHONE_BRANDNAME_FINAL",db_brandnamebhxh,chunksize=200000)
for chunk in df_chunk:
    chunk.to_csv("BHXH.csv",index=False,mode="a")

In [37]:
from IPython.display import clear_output

In [40]:
df_chunk=pd.read_csv("BHXH.csv",chunksize=500000)
count_digit=0
size_df = 0
df_len= pd.Series(dtype="float64")
for chunk in df_chunk:
    df_len = df_len.add(chunk["MABHXH"].astype(str).str.len().value_counts(),fill_value=0)
    size_df+=len(chunk)
    count_digit+=chunk["MABHXH"].map(lambda x:str(x).isdigit()).sum()
    clear_output()
    print("size_df",size_df)
    print("count_digit",count_digit)
    

size_df 27797971
count_digit 27797107


In [41]:
df_len

1           16.0
2            3.0
3            3.0
4            2.0
5            6.0
6          141.0
7            5.0
8            1.0
9            1.0
10    27797763.0
11           7.0
12          23.0
dtype: float64